<div style="direction:rtl;text-align:center"><img src="https://dl.mohammadkh.ir/logo.png" alt="Mohammadkh.ir" style="width: 250px;"/></div>
<h1><div style="direction:rtl;text-align:center">Neural Network</div></h1>

In [1]:
import requests
from lxml import html
#--
import tensorflow as tf
import numpy as np
import os
import time

### create data

In [2]:
# ex:
## dl page html
page = requests.get('https://ganjoor.net/khayyam/robaee/sh1/')
print(page, end='\n\n')

## load tree html
tree = html.fromstring(page.content)

## get robaee in page
tree.xpath('//div[@class="b"]/div/p/text()')

<Response [200]>



['برخیز بتا بیا ز بهر دل ما',
 'حل کن به جمال خویشتن مشکل ما',
 'یک کوزه شراب تا به هم نوش کنیم',
 'زآن پیش که کوزه\u200cها کنند از گل ما']

In [3]:
# # get all robaee khayam
# for i in range(1, 179):
#     page = requests.get('https://ganjoor.net/khayyam/robaee/sh' + str(i) + '/')
#     tree = html.fromstring(page.content)
#     with open('../../__data/khayyam/khayyam.txt', 'a',encoding='utf8') as f:
#         for t in tree.xpath('//div[@class="b"]/div/p/text()'):
#             f.write('|' + str(t.replace('\u200c', ' ')) + '\n')

### Load & preprocess data 

In [2]:
# Load data
text = open('../../__data/Text Generator/khayyam_molana.txt', 'rb').read().decode(encoding='utf-8')

# list all vocabolaries
vocabolaries = sorted(set(text))
# print(vocabolaries)

# encoder decoder
char2index = {u:i for i, u in enumerate(vocabolaries)}
index2char = np.array(vocabolaries)
# print(char2index)

# replace encoder data
text_as_integer = np.array([char2index[c] for c in text])
# print(text_as_integer[:5])

# create tf.data tensor
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_integer)
# for i in char_dataset.take(10):
#     print(index2char[i.numpy()])

# create tensor input data 
sequences = char_dataset.batch(101, drop_remainder=True)
# for i in sequences.take(3):
#     print('--->', ''.join(index2char[i.numpy()]))

# inpot split X,Y
def sit(batch):
    input_text = batch[:-1]
    target_text = batch[1:]
    return input_text, target_text
## map fun
dataset = sequences.map(sit)
# for i in dataset.take(1):
#     print(''.join(index2char[i[0].numpy()]))
#     print(''.join(index2char[i[1].numpy()]))

# df baching
dataset = dataset.batch(128, drop_remainder=True)
print(dataset)

<BatchDataset element_spec=(TensorSpec(shape=(128, 100), dtype=tf.int32, name=None), TensorSpec(shape=(128, 100), dtype=tf.int32, name=None))>


### creat & fit model

In [3]:
vocabolary_size = len(vocabolaries)
embedding_dim = 256
rnn_units = 1024

In [4]:
model = tf.keras.Sequential([
    
    tf.keras.layers.Embedding(vocabolary_size, embedding_dim),
    tf.keras.layers.GRU(rnn_units, return_sequences=True),
    tf.keras.layers.Dense(vocabolary_size)
    
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         12032     
                                                                 
 gru (GRU)                   (None, None, 1024)        3938304   
                                                                 
 dense (Dense)               (None, None, 47)          48175     
                                                                 
Total params: 3,998,511
Trainable params: 3,998,511
Non-trainable params: 0
_________________________________________________________________


In [5]:
# test model random weight
for input_text, target_text in dataset.take(1):
    output = model.predict(input_text)
    # print(output[0])
si = tf.random.categorical(output[0], num_samples=1)
''.join(index2char[tf.squeeze(si, axis=-1).numpy()])  

'شؤةثآث|خَهوزَبةضی!!ش؟حپصنچخعر!گ|اصکةچت رببق|جءحسنٌذ؟سی!بًترهچَّث\nحزپجژطٌَسغةژمء\nزکتشتئنٔ\nٌئٌحلئحلچض!'

In [6]:
def loss_f(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss_f)

In [7]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='../../__data/Text Generator/khayyam_molana_.h5',
                                                monitor='loss', save_best_only=True, save_weights_only=True)

In [9]:
history = model.fit(dataset, epochs=1, verbose=1, callbacks=[checkpoint])

35/35 [==============================] - 42s 1s/step - loss: 0.1023


### generator

In [10]:
model.load_weights('../../__data/Text Generator/khayyam_molana.h5')

##### V1

In [11]:
model.reset_states()
# create input generator
num_generate = 1000
first_string = 'به نام خداوند جان و خرد'
input_eval = [char2index[s] for s in first_string]
input_eval = tf.expand_dims(input_eval, 0)
    
# genereator predict
text_generated = ['به نام خداوند جان و خرد']
for i in range(3):
    # pass input
    predictions = model.predict(input_eval)
    # expand dim output
    predictions = tf.squeeze(predictions, 0)
    
    # get random weighted
    predicted_ids = tf.random.categorical(predictions, num_samples=1).numpy()
    
    # expand dim for input
    input_eval = tf.expand_dims(tf.squeeze(predicted_ids, axis=-1).numpy(), 0).numpy()
    # add to output generator
    text_generated.append(index2char[tf.squeeze(predicted_ids, axis=-1).numpy()])

    
for i in text_generated:
    print(''.join(i))    

به نام خداوند جان و خرد
ر سینواام ند آان و بو 

 چؤاه ین یمامنن ش آهدب|
نهمناشک خکان

دیدطسر در


##### V2

In [12]:
model.reset_states()
# create input generator
num_generate = 1000
first_string = 'به نام خداوند جان و خرد'
input_eval = [char2index[s] for s in first_string]
input_eval = tf.expand_dims(input_eval, 0)

# genereator predict
text_generated = ['به نام خداوند جان و خرد']
for i in range(500):
    # pass input
    predictions = model.predict(input_eval)
    # expand dim output
    predictions = tf.squeeze(predictions, 0)
    
    # get random weighted --> just firest work
    predicted_ids = tf.random.categorical(predictions, num_samples=1).numpy().reshape(-1, 1)[-1][0]
    
    # append to message and create new input
    message = np.append(input_eval[0].numpy(), predicted_ids)[1:]
    # expand dim for input
    input_eval = tf.expand_dims(message, 0)
    # add to output generator
    text_generated.append(index2char[predicted_ids])

    
for i in ''.join(text_generated).split('\n'):
    print(i)    

به نام خداوند جان و خرد
|جمله است این هر دوان کشت از بهر غباب
|از برای حق که گاو در ملا
|کان بخواهد سوی محسوسات ژخل
|گه تنامض لا چو پرهانی بود
|چون بمودی هیچ سگ می روی تو
|وز همر از لقمه مسکن خمر
|درد او پیدا و دست و او ز خاک
|نه کمان تاویل بر گردانیان
|بعد از آن گفتش سلام التقوی است
|لاجرم بندند پیش از قصر ما
|احمدا این کار پیشینی برون
|شد شقی و خوبیات او یک خانه باش
|دست و پاهاشان گواهی می دهند
|تا نماید بار با خود گرد تاست آورد
|ناپذیرف نیست کرد آن شهرها
|نطق کو بدزاده لا محلوم باش
|گوش را بگذار و کی آلود باشد
|چون


##### V3

In [13]:
model.reset_states()
# create input generator
num_generate = 1000
first_string = 'به نام خداوند جان و خرد'
input_eval = [char2index[s] for s in first_string]
input_eval = tf.expand_dims(input_eval, 0)

# genereator predict
text_generated = ['به نام خداوند جان و خرد']
for i in range(500):
    # pass input
    predictions = model.predict(input_eval)
    # expand dim output
    predictions = tf.squeeze(predictions, 0)
    
    # get argmax just firest work
    predicted_ids = np.array(predictions.numpy()).argmax(axis=1).reshape(-1, 1)[-1][0]
    
    # append to message and create new input
    message = np.append(input_eval[0].numpy(), predicted_ids)[1:]
    # expand dim for input
    input_eval = tf.expand_dims(message, 0)
    # add to output generator
    text_generated.append(index2char[predicted_ids])

    
for i in ''.join(text_generated).split('\n'):
    print(i)

به نام خداوند جان و خرد
|جمله مرغان را چو زندانی بود
|چون بدر که عقل خود را در کشید
|تا زنده آن کاری که در دل شک نظر
|با در تا می گزارد هیچ خشم
|زشم شاهنشه که مدخل ای فلان
|من بگویم رست از سوی جهان
|آن دو دزدانیم ابلهست این را ز شرق
|کیست یک خود کی بر آن را منتظر
|با دل موسی مگه کلی از شکر و صد
|چشم آخربین بسسته می شوی گویی تو صد
|همچو اندر شیر خاکی می برد
|من نکرد او ملک هفت زنگ
|این جهان نفس و در آمد نه ز نان
|نیست خاده کرده قودی حال او
|باز گرد امروز فهم باداری
|تا چند کناره و ناپیدا شنو
|این چنین شه نام تستیح اندر


<div class="alert alert-block alert-info">
<div style="direction:rtl;text-align:left"><strong>Neural Network</strong><br>MohammadReza <strong>Khajedaloi</strong><br><br>
</div>
<div style="direction:rtl;text-align:right">
<a href="http://mohammadkh.ir/">WebSite</a> - <a href="https://github.com/khajedaloi/">GitHub</a> - <a href="https://www.linkedin.com/in/mohammad-kh/">Linkedin</a>
</div>
</div>